In [52]:
print('ok')

ok


In [53]:
a = 1
b = 2
c = b + a
c

3

In [54]:
c

3

# 这是标题
[链接](http://www.baidu.como)

In [2]:
a = [i for i in range(100)]
volume = []
while a:
    volume.append(a[:10])
    a = a[10:]
a, volume

([],
 [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
  [10, 11, 12, 13, 14, 15, 16, 17, 18, 19],
  [20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
  [30, 31, 32, 33, 34, 35, 36, 37, 38, 39],
  [40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
  [50, 51, 52, 53, 54, 55, 56, 57, 58, 59],
  [60, 61, 62, 63, 64, 65, 66, 67, 68, 69],
  [70, 71, 72, 73, 74, 75, 76, 77, 78, 79],
  [80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
  [90, 91, 92, 93, 94, 95, 96, 97, 98, 99]])

In [5]:
data = []
for i in range(20):
    data.append({'domain': 'domain{}.com'.format(i), 'count': 100-i})
data

[{'domain': 'domain0.com', 'count': 100},
 {'domain': 'domain1.com', 'count': 99},
 {'domain': 'domain2.com', 'count': 98},
 {'domain': 'domain3.com', 'count': 97},
 {'domain': 'domain4.com', 'count': 96},
 {'domain': 'domain5.com', 'count': 95},
 {'domain': 'domain6.com', 'count': 94},
 {'domain': 'domain7.com', 'count': 93},
 {'domain': 'domain8.com', 'count': 92},
 {'domain': 'domain9.com', 'count': 91},
 {'domain': 'domain10.com', 'count': 90},
 {'domain': 'domain11.com', 'count': 89},
 {'domain': 'domain12.com', 'count': 88},
 {'domain': 'domain13.com', 'count': 87},
 {'domain': 'domain14.com', 'count': 86},
 {'domain': 'domain15.com', 'count': 85},
 {'domain': 'domain16.com', 'count': 84},
 {'domain': 'domain17.com', 'count': 83},
 {'domain': 'domain18.com', 'count': 82},
 {'domain': 'domain19.com', 'count': 81}]

In [19]:
import re

DOMAIN_HOST_RE = re.compile(
     r'^(?=^.{3,255}$)[a-zA-Z0-9][-a-zA-Z0-9]{0,62}(\.[a-zA-Z0-9][-a-zA-Z0-9]{0,62})+$'
)
DOMAIN_HOST_RE.match('www.baidu.wwwsasdfcom')

re.compile('^https?:\/\/[\w.-]+(:\d+)?').match('http://www.bai-du.comx:900')

a = 'a,b'
a.split(','), a

(['a', 'b'], 'a,b')

In [5]:
 
order_data = {}        
args = {
    'to': ['yexy@knownsec.coom'],
    'sub': {
    "company": order_data.get('company', '知道川渝'),
    "contacts": order_data.get('contacts', 'yexy'),
    "phone": order_data.get('phone', '18280081234'),
    "account": order_data.get('account', 'yexy@knownsec.com'),
    "qq": order_data.get('qq', 'asdf'),
    'service_name': 'asdf',
    "domain": order_data.get('domain', 'asdf'),
    "create_time": 'asdfasdf',
}
}
    
dict_string_list = []
for key, value in args.iteritems():
    if isinstance(value, dict):
        dict_list = []
        for k, v in value.iteritems():
            encode_list = ["'{}'".format(s.encode('utf-8'))
                           if isinstance(s, unicode)
                           else str("'{}'".format(s)) for s in v]
            list_str = "'{}'".format(k) +\
                       ':[{}]'.format(','.join(encode_list))
            dict_list.append(list_str)
        value_str = '{' + ','.join(dict_list) + '}'
    else:
        value_str = value.encode('utf-8')\
                    if isinstance(value, unicode) else str(value)
    dict_str = "'{}'".format(key) + ':' + value_str
    dict_string_list.append(dict_str)
args = '{' + ','.join(dict_string_list) + '}'
args        

AttributeError: 'dict' object has no attribute 'iteritems'